In [1]:
from modules import processing_csv_data
from modules.submodules import dis, extracting_coordinates, adding_custom_marker, executing_requete_sparql, _point, queringWikidata
import csv
from rdflib.plugins.sparql import prepareQuery
from rdflib import Graph, URIRef, Literal, Namespace, XSD
import osmnx as ox
import folium

In [2]:
kesaio = Namespace("http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/")
common_core_ns = Namespace("http://www.ontologyrepository.com/CommonCoreOntologies/")
hasRole = URIRef("http://purl.obolibrary.org/obo/RO_0000087")
participatesIn = URIRef("http://purl.obolibrary.org/obo/RO_0000056")

# L'instanciation

In [3]:
rawData = "simulationPlanOrsec/data/dataFromWikidata.csv"
processedData = "simulationPlanOrsec/data/dataFromWikidataNew.csv"
processing_csv_data.process_csv_data(rawData, processedData)
g = Graph()
g.parse("simulationPlanOrsec/Ontologie/file10.rdf")
with open(processedData, "r", newline="", encoding="utf-8") as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        uri_op = dis.catastrophe(g,row)

In [4]:
point = coordonnees = row["coordonnees"].split("/")[-1]
lon, lat = extracting_coordinates.extract_coordinates(point)
print(f"Longitude : {lon}, Latitude : {lat}")

Longitude : 141.933, Latitude : 42.671


In [5]:
carto = (lat, lon)
G = ox.graph_from_point(carto, dist=15000, network_type="drive")
m = ox.plot_graph_folium(G)
tileset_url = 'https://tile.openstreetmap.org/{z}/{x}/{y}.png'
folium.TileLayer(
    tiles=tileset_url,
    max_zoom=19,
    attr='<a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>'
).add_to(m)

In [6]:
icone_path = "/home/mandiaye/LORIA/V2/icone/tremblement-de-terre.png"
custom_icon = folium.CustomIcon(
    icon_image=icone_path,
    icon_size=(30, 30),  
)
adding_custom_marker.add_custom_marker(lon, lat, custom_icon, m, tooltip=row["description"])
m

# PRV (Point de rassemblement des victimes)

In [7]:
point1 = (lat, lon)
point2 = (42.57625, 141.923167)
new_distance = 200 

latPRV, lonPRV = _point.intermediate_point(point1, point2, new_distance)

In [8]:
uri_prv = dis.create_PRV(g, lonPRV, latPRV)

In [9]:
custom_icon = folium.CustomIcon(
    icon_image="/home/mandiaye/LORIA/V2/icone/point-de-rencontre.png",
    icon_size=(32, 32),  
    icon_anchor=(16, 32),  
)
tooltip = "PRV (Point de rassemblement des victimes)"
adding_custom_marker.add_custom_marker(lonPRV, latPRV, custom_icon, m, tooltip=tooltip)

m

# Acte de Ramassage des victimes

In [10]:
uri_actDeRamassage = dis.createRamassage_act(g, uri_prv, uri_op)

In [11]:
nombreMorts = int(row["nombreMorts"]) 
nombreBlesse = int(row["nombreBlesse"])

nombreDeVictime = nombreMorts + nombreBlesse

nombre_pompiers = 0
nombre_policiers = 0
nombre_officiers_samu = 0

if nombreDeVictime <= 9:
    nombre_pompiers = 2
    nombre_policiers = 1
    nombre_officiers_samu = 2
elif 7 <= nombreDeVictime <= 25:
    nombre_pompiers = 5
    nombre_policiers = 2
    nombre_officiers_samu = 5
else:
    nombre_pompiers = 10
    nombre_policiers = 5
    nombre_officiers_samu = 10

In [12]:
dis.create_personnel(g, uri_actDeRamassage, nombre_pompiers, nombre_policiers, nombre_officiers_samu)

###### Écrire et exécuter une requête SPARQL pour obtenir toutes les instances de la classe "Victime" et mettre "ramassé" à True

In [13]:
requete_sparql = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type :Victime.
    }
"""
results = g.query(requete_sparql)
for row in results:
    victime_instance = row.victime
    g.add((uri_actDeRamassage, kesaio["ramasser"], victime_instance))
    g.add((victime_instance, kesaio["ramassé"], Literal(True, datatype=XSD.boolean)))

# Le poste médical avancé (PMA)

In [14]:
establishments = {
    "Hôpital": "Q16917",
    "Police": "Q35535",
    "Gendarmerie": "Q204310",
    "CasernePompier": "Q1195942",
    "École": "Q3914",
    "Université": "Q3918"
}

In [15]:
resultsHopitaux = queringWikidata.search_nearby_entities(lat, lon, establishments["Hôpital"])

##### Hôpitaux proches

In [16]:
for result in resultsHopitaux['results']['bindings']:
    id = result['entity']['value'].split('/')[-1]
    name = result['entityLabel']['value']
    distance = result["dist"]["value"]
    geoLoc = result["location"]["value"]
    print(f"ID Wikidata : {id} - Nom : {name} - Localisation : {geoLoc} - Distance : {distance} km")

In [17]:
for result in resultsHopitaux["results"]["bindings"]:
    longitude, latitude = extracting_coordinates.extract_coordinates(result["location"]["value"])
    name = result['entityLabel']['value']
    distance = result["dist"]["value"]

    custom_icon = folium.CustomIcon(
        icon_image="/home/mandiaye/LORIA/V2/icone/hopital.png",
        icon_size=(32, 32),  
        icon_anchor=(16, 32),  
    )
    adding_custom_marker.add_custom_marker(longitude, latitude, custom_icon, m, tooltip=name)

In [18]:
distance_geo_dict = {}
for result in resultsHopitaux['results']['bindings']:
    distance = float(result["dist"]["value"])
    geo_location = result["location"]["value"]
    if distance > 10:
        distance_geo_dict[distance] = geo_location  

min_distance_above_10 = min([d for d in distance_geo_dict.keys()])
geo_location = distance_geo_dict[min_distance_above_10].split('(')[1].split(')')[0]
latitude, longitude = geo_location.split()

In [19]:
latPMA, lonPMA = _point.intermediate_point(point1, point2, 1000)

In [20]:
uri_pma = dis.create_medical_post(g, lonPMA, latPMA)

In [21]:
custom_icon = folium.CustomIcon(
    icon_image="/home/mandiaye/LORIA/V2/icone/pma.png",
    icon_size=(32, 32),  
    icon_anchor=(16, 32),  
)
tooltip="Le poste médical avancé (PMA)"
adding_custom_marker.add_custom_marker(lonPMA, latPMA, custom_icon, m, tooltip=tooltip)
m

### Acte de secours des victimes

In [22]:
uri_actDeSoins = dis.create_medical_aid(g, uri_pma, uri_op)

##### Récupérer les victimes à soigner

In [23]:
requete_sparql = """
    SELECT (COUNT(?victime) as ?blessesASoigner) 
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ur)
    }
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for row in resultats:
    blessesASoigner = row.blessesASoigner

blessesASoigner = int(blessesASoigner)
print(blessesASoigner)

2


###### Calculez combien de médecins et d'infirmiers sont nécessaires

In [24]:
nombre_de_medecins = blessesASoigner // 5
nombre_d_infirmiers = blessesASoigner // 5

if blessesASoigner % 5 != 0:
    nombre_de_medecins += 1
    nombre_d_infirmiers += 1

print("Nombre total de victimes :", blessesASoigner)
print("Nombre de médecins nécessaires :", nombre_de_medecins)
print("Nombre d'infirmiers nécessaires :", nombre_d_infirmiers)

Nombre total de victimes : 2
Nombre de médecins nécessaires : 1
Nombre d'infirmiers nécessaires : 1


In [25]:
dis.create_medical_staff(g, uri_actDeSoins, nombre_de_medecins, nombre_d_infirmiers)

###### Écrire et exécuter une requête SPARQL pour obtenir toutes les instances de la classe "Victime" et mettre "soigné" à True

In [26]:
requete_sparql = """
    SELECT ?blessesASoigner
    WHERE {
        ?blessesASoigner rdf:type kesaio:Victime.
        ?blessesASoigner kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ur)
    }
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for row in resultats:
    blessesASoigner = row.blessesASoigner
    g.add((uri_actDeSoins, kesaio["soigner"], blessesASoigner))
    g.add((blessesASoigner, kesaio["soigné"], Literal(True, datatype=XSD.boolean)))

##### Suivi psycho medical

###### Récupérer les victimes souffrant d'un traumatisme psychologique

In [27]:
requete_sparql = """
    SELECT (COUNT(?victime) as ?suivi_psycho_medical) 
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ump)
    }
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio)
for row in resultats:
    suivi_psycho_medical = row.suivi_psycho_medical

suivi_psycho_medical = int(suivi_psycho_medical)
print(suivi_psycho_medical)

1


In [28]:
nombre_psychologues = suivi_psycho_medical // 5
if suivi_psycho_medical % 5 != 0:
    nombre_psychologues += 1

In [29]:
dis.create_other_staff(g, uri_actDeSoins, nombre_psychologues)

In [30]:
requete_sparql = """
    SELECT ?suivi_psycho_medical
    WHERE {
        ?suivi_psycho_medical rdf:type kesaio:Victime.
        ?suivi_psycho_medical kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ump)
    }
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio)
for row in resultats:
    suivi_psycho_medical = row.suivi_psycho_medical
    g.add((uri_actDeSoins, kesaio["soigner"], suivi_psycho_medical))
    g.add((suivi_psycho_medical, kesaio["soigné"], Literal(True, datatype=XSD.boolean)))

##### Gestion des morts

In [31]:
nombre_pompiers = nombreMorts // 5

In [32]:
dis.create_PompierPourLesMorts(g, uri_actDeSoins, nombre_pompiers)

In [33]:
requete_sparql = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:décédé)
    }
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio)
for row in resultats:
    victime_instance = row.victime
    g.add((victime_instance, kesaio["deposerAlaMorgue"], Literal(True, datatype=XSD.boolean)))

##### Gestion des rescapés

In [34]:
requete_sparql = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:indemne)
    }
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio)
for row in resultats:
    victime_instance = row.victime
    g.add((victime_instance, kesaio["relogé"], Literal(True, datatype=XSD.boolean)))

### Acte d'évacuation des victimes

##### Récupérer les victimes à évacuer

In [35]:
uri_actDEvacution = dis.create_evacuation_act(g,uri_pma, uri_op)

In [36]:
requete_sparql = """
    SELECT (COUNT(?victime) as ?blessesAEvacuer) 
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ua)
    }
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio)
for row in resultats:
    blessesAEvacuer = row.blessesAEvacuer

blessesAEvacuer = int(blessesAEvacuer)
print(blessesAEvacuer)

2


In [37]:
nombre_ambulance = blessesAEvacuer // 5
nombre_helicoptere = blessesAEvacuer // 10

if blessesAEvacuer % 5 != 0:
    nombre_ambulance += 1

In [38]:
dis.add_ambulance_and_helicopter(g, uri_actDEvacution, nombre_ambulance, nombre_helicoptere)

In [39]:
requete_sparql = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ua)
    }
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio)
for row in resultats:
    victime_instance = row.victime
    g.add((uri_actDEvacution, kesaio["évacuer"], victime_instance))
    g.add((victime_instance, kesaio["évacué"], Literal(True, datatype=XSD.boolean)))

In [40]:
shortest_paths = []
orig_node = ox.distance.nearest_nodes(G, X=point1[1], Y=point1[0])
G = ox.speed.add_edge_speeds(G)
G = ox.speed.add_edge_travel_times(G)
for result in resultsHopitaux['results']['bindings']:
    geoLoc = result["location"]["value"]
    lon, lat = map(float, geoLoc[6:-1].split(" "))  
    dest_node = ox.distance.nearest_nodes(G, Y=lat, X=lon)
    route = ox.shortest_path(G, orig_node, dest_node, weight="travel_time")
    shortest_paths.append(route)

fig, ax = ox.plot_graph_routes(G, routes=shortest_paths, 
                               route_colors=["r", "y", "b"], 
                               route_linewidth=6, node_size=1)

##### Enregistrez l'ontologie enrichie dans un fichier RDF 

In [41]:

g.serialize("/home/mandiaye/LORIA/V2/monOnto/ontologie_enrichie.rdf", format="xml")

<Graph identifier=N2732c6a4fe1b4604b928d0a150d3fb17 (<class 'rdflib.graph.Graph'>)>